In [1]:
from pyspark.sql import SparkSession
from functools import reduce
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("CH 09 - series to series UDF").getOrCreate()

23/06/11 09:57:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
gsod = (
    reduce(
        lambda x, y: x.unionByName(y, allowMissingColumns=True),
        [
            spark.read.parquet(f"/opt/spark/data/gsod_noaa/gsod{year}.parquet")
            for year in range(2019, 2021)
        ],
    )
    .dropna(subset=["year", "mo", "da", "temp"])
    .where(F.col("temp") != 9999.9)
    .drop("date")
)

In [4]:
gsod.show(5)

23/06/11 09:57:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+-----+----+---+---+----+----------+------+----------+------+---------+-----+---------+-----+-----------+----+----------+-----+-----+----+--------+----+--------+----+---------+-----+---+------------+----------------+----+-------+--------------------+
|   stn| wban|year| mo| da|temp|count_temp|  dewp|count_dewp|   slp|count_slp|  stp|count_stp|visib|count_visib|wdsp|count_wdsp|mxpsd| gust| max|flag_max| min|flag_min|prcp|flag_prcp| sndp|fog|rain_drizzle|snow_ice_pellets|hail|thunder|tornado_funnel_cloud|
+------+-----+----+---+---+----+----------+------+----------+------+---------+-----+---------+-----+-----------+----+----------+-----+-----+----+--------+----+--------+----+---------+-----+---+------------+----------------+----+-------+--------------------+
|041680|99999|2019| 02| 19|16.1|        15|9999.9|         0|1005.8|       15|999.9|        0|999.9|          0| 8.1|        15| 22.9|999.9|23.0|       *|12.0|    null| 0.0|        I|999.9|  0|           0|               0|   

In [5]:
@F.pandas_udf(T.DoubleType())
def f_to_c(degrees: pd.Series) -> pd.Series:
    return (degrees - 32) * 5 / 9

In [6]:
gsod = gsod.withColumn("temp_c", f_to_c(F.col("temp")))
gsod.select("temp", "temp_c").distinct().show(5)

+----+-------------------+
|temp|             temp_c|
+----+-------------------+
|29.6|-1.3333333333333326|
|53.5| 11.944444444444445|
|70.4| 21.333333333333336|
|85.9| 29.944444444444443|
|37.2| 2.8888888888888906|
+----+-------------------+
only showing top 5 rows



In [9]:
spark.stop()